In [1]:
from modeling_amt.act_utils import ACT_basic, gen_timing_signal
import torch
from torch import nn

In [10]:
class Model(nn.Module):
    def __init__(self, hidden_size, num_layers, max_length):
        super().__init__()
        self.emb = nn.Embedding(10, hidden_size)
        self.block = nn.TransformerEncoderLayer(d_model=hidden_size, dim_feedforward=hidden_size * 4, nhead=4, batch_first=True)
        self.act = ACT_basic(hidden_size)
        self.time_enc = gen_timing_signal(max_length, hidden_size)
        self.pos_enc = gen_timing_signal(num_layers, hidden_size)
    def forward(self, x):
        x = self.emb(x)
        mask = nn.Transformer.generate_square_subsequent_mask(x.size(-2))
        x = self.act(x, x, lambda x: self.block(x, src_mask=mask, is_causal=True), self.time_enc, self.pos_enc, max_hop=4)
        return x

model = Model(16, 10, 100).to('cuda')
x = torch.randn((5, 6, 16))
t = torch.randint(0, 10, (5, 6)).to('cuda')
mask = nn.Transformer.generate_square_subsequent_mask(x.size(-2))
out, state = model(t)


In [11]:
out.shape

torch.Size([5, 6, 16])

In [12]:
state

(tensor([[0.1853, 0.3178, 0.1547, 0.1518, 0.1211, 0.1646],
         [0.1667, 0.3178, 0.2685, 0.1139, 0.2183, 0.1311],
         [0.3733, 0.1353, 0.1594, 0.1242, 0.1320, 0.1646],
         [0.2404, 0.1984, 0.1097, 0.1518, 0.1320, 0.1427],
         [0.2006, 0.1353, 0.1339, 0.2067, 0.1211, 0.1427]], device='cuda:0',
        grad_fn=<AddBackward0>),
 tensor([[2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.]], device='cuda:0'))

In [13]:
from datasets import load_dataset

dataset = load_dataset('mbur/1dCA_r2s20T20')
dataset

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/950000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['rule', 't=0', 't=1', 't=2', 't=3', 't=4', 't=5', 't=6', 't=7', 't=8', 't=9', 't=10', 't=11', 't=12', 't=13', 't=14', 't=15', 't=16', 't=17', 't=18', 't=19'],
        num_rows: 950000
    })
    validation: Dataset({
        features: ['rule', 't=0', 't=1', 't=2', 't=3', 't=4', 't=5', 't=6', 't=7', 't=8', 't=9', 't=10', 't=11', 't=12', 't=13', 't=14', 't=15', 't=16', 't=17', 't=18', 't=19'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['rule', 't=0', 't=1', 't=2', 't=3', 't=4', 't=5', 't=6', 't=7', 't=8', 't=9', 't=10', 't=11', 't=12', 't=13', 't=14', 't=15', 't=16', 't=17', 't=18', 't=19'],
        num_rows: 100000
    })
})

In [19]:
cs = [c for c in dataset['train'][0]['t=0']]
cs

['1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0']